# Pandas

Pandas est une librairie Python dédiée à l'analyse de données.

## Series

La structure de données Series permet de gérer une **table de données à deux colonnes**, dans laquelle :
- les données sont ordonnées
- la première colonne contient une clé (index)
- le deuxième colonne contient des valeurs
- la deuxième colonne porte un nom

On peut initialiser une structure Series **à partir d'une liste** de valeurs. Dans ce cas, Pandas affecte automatiquement un index numérique à chaque valeur en partant de zéro.

In [1]:
import pandas as pd

In [2]:
animaux = ["chien", "chat", "lapin"]
pd.Series(animaux)

0    chien
1     chat
2    lapin
dtype: object

On note que dans ce cas le type de données est object.

In [3]:
nombres = [10,4,8]
ns = pd.Series(nombres)
ns

0    10
1     4
2     8
dtype: int64

On note que dans ce cas le type de données est int64.

La structure Series stocke les données sous la forme d'un **tableau Numpy typé**, ce qui lui donne un avantage en termes de performances par rapport à une liste.

In [4]:
nombres = [10,4,None]
pd.Series(nombres)

0    10.0
1     4.0
2     NaN
dtype: float64

On note que l'absence de valeur None est convertie en **np.nan** dans un tableau numérique.

On teste la présence de la valeur NaN de la manière suivante :

In [5]:
import numpy as np
np.isnan(np.nan)

True

Un structure Séries peut également être initialisée **à partir d'un dictionnaire** : dans ce cas, Pandas utilise les clés du dictionnaire pour intialiser les index. 

In [6]:
personne = { "nom":"Dupont", "prénom":"Jean", "age":40 }
s = pd.Series(personne)
s

nom       Dupont
prénom      Jean
age           40
dtype: object

La propriété **index** permet d'accéder aux index d'une structure Series :

In [7]:
s.index

Index(['nom', 'prénom', 'age'], dtype='object')

Les index peuvent être également initialisés en passant une  liste en tant que paramètre nommé du constructeur :

In [8]:
pd.Series(["Dupont","Jean",40], index=["nom","prénom","age"])

nom       Dupont
prénom      Jean
age           40
dtype: object

On peut accéder aux valeurs stockées dans une structure Series :
- par leur position : propriété **iloc**[position]
- par leur index (clé) : propriété **loc**[clé]

In [9]:
s.iloc[2]

40

In [10]:
s.loc["nom"]

'Dupont'

## Performances

On peut parcourir les valeurs d'une structure Series et en calculer la somme explicitement :

In [11]:
somme = 0
for num in ns:
    somme = somme + num
    
somme

22

Mais cette méthode est lente, car elle ne tire pas partie des capacités de **calcul parallèle** des ordinateurs modernes.

Numpy et Pandas définissent des méthodes applicables directement à leurs structures de données qui sont optimisées pour réaliser les opérations en parallèle :

In [12]:
total = np.sum(ns)
total

22

L'exemple ci-dessous va illustrer comment **mesurer la différence de performance** entre ces deux techniques :

In [13]:
# Création d'une série de 100 000 éléments
s = pd.Series(np.random.randint(0,100,100000))

# La méthode head() permet d'afficher les 5 premiers éléments
s.head()

0    69
1    28
2    61
3    13
4    15
dtype: int32

In [14]:
len(s)

100000

La directive **%%timeit** permet de mesurer le temps d'exécution d'une cellule du notebook. Elle prend pour paramètre le nombre de fois où on souhaite réexécuter le fragment de code avant de prendre la moyenne des temps d'exécution :

In [15]:
%%timeit -n 10
somme = 0
for num in s:
    somme += num

33.4 ms ± 7.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit -n 10
somme = np.sum(s)

401 µs ± 117 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


On voit que la deuxième méthode utilisant les fonctionnalités de Numpy est environ **100 fois plus rapide** sur la machine de test (dépend des capacités de la machine).

On ne devrait donc **jamais coder de parcours explicite des éléments** d'un tableau Numpy ou d'une structure Pandas dans le code (boucle for ou while).

## Différences par rapport à une base de données relationnelle

Les index (clés) comme les valeurs peuvent être de **types différents** dans une même structure Series :

In [17]:
mixed = pd.Series([1,2,3])
mixed.loc["animal"] = "chien"
mixed

0             1
1             2
2             3
animal    chien
dtype: object

Les index (clés) ne sont **pas obligatoirement uniques** et peuvent être répétés :

In [18]:
repeat = pd.Series(["chien","rose","chat"],index=["animal","fleur","animal"])
repeat

animal    chien
fleur      rose
animal     chat
dtype: object

Dans ce cas, le résultat d'une requête sur une structure Series n'est pas une valeur mais à nouveau une structure Series :

In [19]:
repeat.loc["animal"]

animal    chien
animal     chat
dtype: object

In [20]:
repeat.loc["fleur"]

'rose'

Les opérations réalisées sur une structure Series ne **modifient pas la structure originale**, mais retournent un nouvel objet. 

Exemple avec la méthode **append()** :

In [21]:
s1 = pd.Series([1,2,3])
s2 = pd.Series([4,5,6])
s1.append(s2)

0    1
1    2
2    3
0    4
1    5
2    6
dtype: int64

In [22]:
s1

0    1
1    2
2    3
dtype: int64

## DataFrame

La structure **DataFrame est une table de données à deux dimensions**, dans laquelle chaque ligne a un index (clé), et chaque colonne a un nom.

La liste des index (clés) est accessible par la propriété **index**, la liste des noms de colonnes est accessible par la propriété **columns**.

Comme dans la structure Series, les **propriétés loc[] et iloc[]** permettent d'accéder aux lignes par index ou par position.

L'opérateur d'indexation (crochets) permet d'accéder à une valeur particulière d'une ligne à partir du nom de colonne.

On peut créer une DataFrame à partir de :
- une **liste de Series** où chaque Series représente une ligne de donnée
- une **liste de dictionnaires** où chaque dictionnaire représente une ligne de données

In [23]:
achat1 = pd.Series({"nom":"Jean","article":"pain","prix":1.1})
achat2 = pd.Series({"nom":"Pierre","article":"lait","prix":2.5})
achat3 = pd.Series({"nom":"Marc","article":"chips","prix":1.9})
df = pd.DataFrame([achat1,achat2,achat3],index=["magasin1","magasin1","magasin2"])
df

,nom,article,prix
magasin1,Jean,pain,1.1
magasin1,Pierre,lait,2.5
magasin2,Marc,chips,1.9


La sélection d'une ligne est réalisée à l'aide de la propriété loc :

In [24]:
df.index

Index(['magasin1', 'magasin1', 'magasin2'], dtype='object')

In [25]:
df.loc["magasin2"]

nom         Marc
article    chips
prix         1.9
Name: magasin2, dtype: object

In [26]:
type(df.loc["magasin2"])

pandas.core.series.Series

In [27]:
df.loc["magasin1"]

,nom,article,prix
magasin1,Jean,pain,1.1
magasin1,Pierre,lait,2.5


In [28]:
type(df.loc["magasin1"])

pandas.core.frame.DataFrame

La sélection d'une colonne se fait simplement par son nom :

In [29]:
df.columns

Index(['nom', 'article', 'prix'], dtype='object')

In [30]:
df["article"]

magasin1     pain
magasin1     lait
magasin2    chips
Name: article, dtype: object

In [31]:
df["nom"]

magasin1      Jean
magasin1    Pierre
magasin2      Marc
Name: nom, dtype: object

Il est recommandé de sélectionner une valeur de la table de la manière suivante

In [32]:
df.loc["magasin2","article"]

'chips'

On peut également chaîner une sélection de ligne, puis une sélection de colonne, mais il faut se rappeler qu'une nouvelle structure Series ou DataFrame est créée à chaque appel, ce qui est inefficace en lecture et conduit à des erreurs en écriture (la structure originale n'est pas modifiée comme on s'y attend). 

Le **chaînage est donc à éviter** avec Pandas.

In [33]:
df.loc["magasin2"]["article"]

'chips'

In [34]:
df.loc["magasin2"]["article"] = "alumettes"
df.loc["magasin2"]["article"]

C:\Users\laure\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'chips'

La propriété **T** permet d'accéder à une version transposée du tableau, qui échange les colonnes et les lignes :

In [35]:
df.T

,magasin1,magasin1,magasin2
nom,Jean,Pierre,Marc
article,pain,lait,chips
prix,1.1,2.5,1.9


In [36]:
df.T.loc["article"]

magasin1     pain
magasin1     lait
magasin2    chips
Name: article, dtype: object

La sélection des lignes et colonnes d'une DataFrame supporte la syntaxe de slicing :

In [37]:
df.loc["magasin2":,["nom","prix"]]

,nom,prix
magasin2,Marc,1.9


## Opérations sur la structure DataFrame

La méthode **drop()** permet de supprimer une ligne désignée par son index. Attention, comme indiqué précédemment, cette méthode retourne une nouvelle structure avec une ligne en moins, et ne modifie pas la structure originale.

In [38]:
df.drop("magasin2")

,nom,article,prix
magasin1,Jean,pain,1.1
magasin1,Pierre,lait,2.5


In [39]:
df

,nom,article,prix
magasin1,Jean,pain,1.1
magasin1,Pierre,lait,2.5
magasin2,Marc,chips,1.9


Le paramètre **inplace** permet de modifier directement la structure originale :

In [40]:
dfc = df.copy()
dfc.drop("magasin2",inplace=True)
dfc

,nom,article,prix
magasin1,Jean,pain,1.1
magasin1,Pierre,lait,2.5


Le paramètre **axis** permet de supprimer une colonne :

In [41]:
dfc.drop("nom",axis=1)

,article,prix
magasin1,pain,1.1
magasin1,lait,2.5


On ajoute une nouvelle colonne simplement en lui affectant une valeur :

In [42]:
dfc["quantité"] = None
dfc

,nom,article,prix,quantité
magasin1,Jean,pain,1.1,None
magasin1,Pierre,lait,2.5,None


On peut modifier une colonne en masse à l'aide des opérateurs vus dans la section Performances. Par exemple, pour appliquer une réduction de prix de 20%, on peut écrire :

In [43]:
df["prix"] *= 0.8
df

,nom,article,prix
magasin1,Jean,pain,0.88
magasin1,Pierre,lait,2.00
magasin2,Marc,chips,1.52
